# Tetrads for Evaluating $\psi_4$

### Authors: Patrick Nelson & Zach Etienne



<a id='initializenrpy'></a>

# Step 1: Initialize core NRPy+ modules \[Back to [top](#toc)\]
$$\label{initializenrpy}$$

In [1]:
# Step 1.a: import all needed modules from NRPy+:
import sympy as sp
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
import reference_metric as rfm

# Step 1.b: Set the coordinate system for the numerical grid
par.set_parval_from_str("reference_metric::CoordSystem","Spherical")

# Step 1.c: Given the chosen coordinate system, set up 
#           corresponding reference metric and needed
#           reference metric quantities
# The following function call sets up the reference metric
#    and related quantities, including rescaling matrices ReDD,
#    ReU, and hatted quantities.
rfm.reference_metric()

# Step 1.d: Set spatial dimension (must be 3 for BSSN, as BSSN is 
#           a 3+1-dimensional decomposition of the general 
#           relativistic field equations)
DIM = 3

# Step 1.e: Import all ADM quantities as written in terms of BSSN quantities
import BSSN.ADM_in_terms_of_BSSN as AB
AB.ADM_in_terms_of_BSSN()

# Step 1.f: Initialize TetradChoice parameter
thismodule = __name__
# Current option: QuasiKinnersley = choice made in Baker, Campanelli, and Lousto. PRD 65, 044001 (2002)
par.initialize_param(par.glb_param("char", thismodule, "TetradChoice", "QuasiKinnersley"))

initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::M_PI
initialize_param() minor warning: Did nothing; already initialized parameter reference_metric::RMAX


# Step 2: The quasi-Kinnersley tetrad

To define the Weyl scalars, first a tetrad must be chosen. Below, for compatibility with the [WeylScal4 diagnostic module](https://bitbucket.org/einsteintoolkit/einsteinanalysis/src/master/WeylScal4/), we implement the quasi-Kinnersley tetrad of [Baker, Campanelli, Lousto (2001)](https://arxiv.org/pdf/gr-qc/0104063.pdf).

We begin with the vectors given in eqs. 5.6 and 5.7 of the BCL paper, which are orthogonal to each other in flat spacetime; one is in the $\phi$ direction, one is in $r$, and the third is the cross product of the first two:
\begin{align}
    v_1^a &= [-y,x,0] \\
    v_2^a &= [x,y,z] \\
    v_3^a &= {\rm det}(\gamma)^{1/2} \gamma^{ad} \epsilon_{dbc} v_1^b v_2^c,
\end{align}

To evaluate the above, we will need the Levi-Civita symbol $\epsilon_{dbc}$, 

In [2]:
# Step 2.a: Declare the Cartesian x,y,z as input parameters
#           and v_1^a, v_2^a, and v_3^a tetrads, 
#           as well as detgamma and gammaUU from 
#           BSSN.ADM_in_terms_of_BSSN
x,y,z = par.Cparameters("REAL",thismodule,["x","y","z"])

v1U = ixp.zerorank1()
v2U = ixp.zerorank1()
v3U = ixp.zerorank1()

detgamma = AB.detgamma
gammaUU  = AB.gammaUU

# Step 2.b: Define the rank-3 version of the Levi-Civita symbol. Amongst
#         other uses, this is needed for the construction of the approximate 
#         quasi-Kinnersley tetrad.
def define_LeviCivitaSymbol_rank3(DIM=-1):
    if DIM == -1:
        DIM = par.parval_from_str("DIM")

    LeviCivitaSymbol = ixp.zerorank3()

    for i in range(DIM):
        for j in range(DIM):
            for k in range(DIM):
                # From https://codegolf.stackexchange.com/questions/160359/levi-civita-symbol :
                LeviCivitaSymbol[i][j][k] = (i - j) * (j - k) * (k - i) / 2
    return LeviCivitaSymbol

# Step 2.c: define v1U and v2U
v1U = [-y,x,sp.sympify(0)]
v2U = [x,y,z]

# Step 2.d: define v3U
LeviCivitaSymbolDDD = define_LeviCivitaSymbol_rank3(DIM=3)
for a in range(DIM):
    for b in range(DIM):
        for c in range(DIM):
            for d in range(DIM):
                v3U[a] += sp.sqrt(detgamma)*gammaUU[a][d]*LeviCivitaSymbolDDD[d][b][c]*v1U[b]*v2U[c]

As our next step, we carry out the Gram-Schmidt orthonormalization process. The vectors $v_i^a$ are placeholders in the code; the final product of the orthonormalization is the vectors $e_i^a$. So,
\begin{align}
e_1^a &= \frac{v_1^a}{\omega_{11}} \\
e_2^a &= \frac{v_2^a - \omega_{12} e_1^a}{\omega_{22}} \\
e_3^a &= \frac{v_3^a - \omega_{13} e_1^a - \omega_{23} e_2^a}{\omega_{33}}, \\
\end{align}
where $\omega_{ij} = v_i^a v_j^b \gamma_{ab}$:

In [3]:
# Step 2.e: Define omega_{ij}
omegaDD = ixp.zerorank2()
gammaDD = AB.gammaDD
def v_vectorDU(v1U,v2U,v3U,  i,a):
    if i==0:
        return v1U[a]
    elif i==1:
        return v2U[a]
    elif i==2:
        return v3U[a]
    else:
        print("ERROR: unknown vector!")
        exit(1)
        
for i in range(DIM):
    for j in range(DIM):
        for a in range(DIM):
            for b in range(DIM):
                omegaDD[i][j] += v_vectorDU(v1U,v2U,v3U, i,a)*v_vectorDU(v1U,v2U,v3U, j,b)*gammaDD[a][b]

# Step 2.f: Define e^a_i. Note that:
#           omegaDD[0][0] = \omega_{11} above; 
#           omegaDD[1][1] = \omega_{22} above, etc.
e1U = ixp.zerorank1()
e2U = ixp.zerorank1()
e3U = ixp.zerorank1()
for a in range(DIM):
    e1U[a] = v1U[a]/omegaDD[0][0]
    e2U[a] = (v2U[a] - omegaDD[0][1]*e1U[a])/omegaDD[1][1]
    e3U[a] = (v3U[a] - omegaDD[0][2]*e1U[a] - omegaDD[1][2]*e2U[a])/omegaDD[2][2]

Once we have orthogonal, normalized vectors, we can construct the tetrad itself, again drawing on eqs. 5.6. We can draw on SymPy's built-in tools for complex numbers to build the complex vector $m^a$:
\begin{align}
    l^a &= \frac{1}{\sqrt{2}} e_2^a \\
    n^a &= -\frac{1}{\sqrt{2}} e_2^a \\
    m^a &= \frac{1}{\sqrt{2}} (e_3^a + i e_1^a) \\
    \implies \Re(m^a) &= \frac{1}{\sqrt{2}} e_3^a \\
             \Im(m^a) &= \frac{1}{\sqrt{2}} e_1^a
\end{align}

We will also assume that $n^0 = l^0 = \frac{1}{\sqrt{2}}$ and that $m^0 = 0$. This last assumption in particular will significantly reduce the terms needed to find $\psi_4$.

In [4]:
# Step 2.g: Construct l^a, n^a, and m^a
isqrt2 = 1/sp.sqrt(2)
l4U = ixp.zerorank1(DIM=4)
n4U = ixp.zerorank1(DIM=4)
mre4U  = ixp.zerorank1(DIM=4)
mim4U  = ixp.zerorank1(DIM=4)
l4U[0] = isqrt2
n4U[0] = isqrt2
mre4U[0] = sp.sympify(0)
mim4U[0] = sp.sympify(0)
for a in range(DIM):
    l4U[a+1] =  isqrt2 * e2U[a]
    n4U[a+1] = -isqrt2 * e2U[a]
    mre4U[a+1] =  isqrt2 * e3U[a]
    mim4U[a+1] =  isqrt2 * e1U[a]